# Workshop 2: Data Cleaning and Feature Engineering

## Business Context: From Raw Transactions to Customer Intelligence

In Workshop 1, we discovered our data has problems: missing values, negative quantities, and mathematical inconsistencies. The marketing team can't use this data yet.

**The challenge:** Raw transaction data shows *what* happened, but marketing needs to know *who* the customer is. They need a **Customer 360** view - a single row per customer with their complete profile.

---

## What We're Building: Customer 360

**Input:** 10,000+ individual transactions
```
order_id | customer_id | product | quantity | amount | date
1001     | C123        | Widget  | 2        | 50.00  | 2024-01-15
1002     | C123        | Gadget  | 1        | 30.00  | 2024-01-20
...
```

**Output:** One row per customer with aggregated features
```
customer_id | total_spend | order_count | recency | tenure | segment
C123        | 580.00      | 8           | 15      | 365    | Premium
```

**This enables:**
- Marketing to identify who is a Premium customer
- ML model to learn patterns: "What makes someone Premium?"
- Future predictions: "Will this new customer become Premium?"

---

## RFM Analysis: The Marketing Standard

We'll use **RFM (Recency, Frequency, Monetary)** - a proven customer segmentation technique:

| Metric | Definition | Why It Matters |
|--------|------------|----------------|
| **Recency** | Days since last purchase | Recent buyers are more likely to buy again |
| **Frequency** | Number of orders | Frequent buyers are loyal |
| **Monetary** | Total spend | High spenders are valuable |

**Example insights:**
- Customer with low recency (5 days) + high frequency (20 orders) = Loyal, Active → Premium
- Customer with high recency (180 days) + low frequency (1 order) = At risk of churn → Basic

---

## Context and Requirements

- **Workshop:** Customer Segmentation for RetailMax
- **Notebook type:** Hands-on Exercise
- **Prerequisites:** `01_Workshop_Data_Exploration.ipynb` completed
- **Technical requirements:**
  - Databricks Runtime 14.x LTS or newer
  - Unity Catalog enabled
- **Execution time:** ~30 minutes

---

## Data Cleaning Strategy

| Issue Type | Strategy | Business Reason |
|------------|----------|-----------------|
| Missing key IDs | Remove row | Can't identify customer → useless |
| Missing descriptive fields | Impute default | Email unknown but customer is real |
| Invalid values | Filter out | Negative quantity = data error |
| Duplicates | Remove | Avoids double-counting spend |

---

## Data Leakage Warning

> **Critical Rule:** When calculating imputation statistics (mean, median) or any transformation parameters, use ONLY training data. Applying statistics calculated from test data leaks future information into the model.

| Correct | Incorrect |
|---------|-----------|
| Calculate mean on train set | Calculate mean on full dataset |
| Apply train mean to test set | Calculate separate mean for test |
| Fit scaler on train only | Fit scaler on all data |

In this workshop, we work with raw data before splitting. The Pipeline in Workshop 03 handles this correctly by fitting transformers only on training data.

---

In [0]:
%run ../demo/00_Setup

## Section 1: Load Data

**Current state:** Raw transaction data with issues identified in Workshop 1

In [0]:
# 1. Load Data
df = spark.table("workshop_sales_data")

## Section 2: Data Cleaning

**Business context:** We can't build Customer 360 on dirty data. First, we clean.

**Issues found in Workshop 1:**
- ❌ Negative quantities (`quantity <= 0`) - likely returns or errors
- ❌ Missing customer IDs - can't aggregate without identity
- ❌ Duplicate transactions - would inflate spending totals

**Goal:** Create a "Silver" layer - clean, validated data ready for analysis.

In [0]:
from pyspark.sql.functions import col, count, when, lit, max, datediff, current_date, to_date

# Exercise 1: Filter invalid records
# Remove rows where 'quantity' <= 0 OR 'total_amount' < 0
# Save result to 'df_clean'
# Compare row counts before and after

df_clean = # TODO: Filter invalid records
# print(f"Original: {df.count()}, Clean: {df_clean.count()}")

## Section 3: Handle Missing Values

**Business context:** Not all missing data is equal. Strategy depends on business impact:

| Column | If Missing | Business Decision |
|--------|------------|-------------------|
| `customer_id` | Can't identify customer | DELETE - unusable |
| `order_id` | Can't track transaction | DELETE - unusable |
| `email` | Customer prefers privacy | IMPUTE - customer is still real |

**Key insight:** Deleting rows loses data. Only delete when absolutely necessary.

In [0]:
# Exercise 2: Handle missing values
# a) Remove rows where 'customer_id' OR 'order_id' is NULL
# b) Fill missing 'email' values with "unknown@example.com"
# Hint: Use .dropna() and .fillna()

df_clean = # TODO: Handle nulls

# print(f"Count after null handling: {df_clean.count()}")

In [0]:
# Exercise 3: Deduplication
# Check for and remove duplicate rows (exact duplicates)
# Hint: Use .dropDuplicates()

df_clean = # TODO: Remove duplicates
# print(f"Count after deduplication: {df_clean.count()}")

### Missing Flags (Informative Missingness)

Sometimes the fact that data is missing is informative. For example:
- A customer without email may be privacy-conscious
- A missing phone number may indicate online-only customer

Creating a "missing flag" allows the model to learn from these patterns.

In [0]:
# Exercise 3b (Optional): Create missing flag for email
# Create a column 'email_missing' that is 1 if email was NULL, 0 otherwise
# This should be done BEFORE filling nulls

# TODO: Create missing flag
# df_clean = df_clean.withColumn("email_missing", when(col("email").isNull(), 1).otherwise(0))

## Section 4: Feature Engineering - Customer 360

**This is the core transformation.** We convert many transactions → one customer profile.

**Why?**
- ML models need one row per prediction target
- We predict segment *per customer*, not per transaction
- Marketing acts on *customers*, not individual orders

**Planned features (RFM + Demographics):**

| Feature | Type | Formula | Business Meaning |
|---------|------|---------|------------------|
| `total_spend` | Monetary | SUM(total_amount) | Customer lifetime value |
| `order_count` | Frequency | COUNT(orders) | Purchase frequency |
| `recency` | Recency | Days since last purchase | Engagement level |
| `tenure` | Demographics | Days since registration | Customer maturity |

**Example aggregation:**
```
Customer C123: 
  - Order 1: $50 on Jan 15
  - Order 2: $30 on Jan 20
  - Order 3: $100 on Feb 1 (last purchase)
  
→ total_spend = $180, order_count = 3, recency = 30 days (if today is March 1)
```

In [0]:
from pyspark.sql.functions import sum, count, avg, min, max, datediff, lit

# Helper: Find reference date (max date in dataset) for Recency calculation
max_date = df_clean.select(max("order_datetime")).collect()[0][0]
print(f"Reference Date: {max_date}")

# Exercise 4: Aggregate data
# Group by 'customer_id', 'customer_segment', 'country', 'registration_date'
# Calculate:
# - total_spend (sum of total_amount)
# - order_count (count of order_id)
# - last_purchase_date (max of order_datetime)

customer_features = df_clean.groupBy("customer_id", "customer_segment", "country", "registration_date") \
    .agg(
        # TODO: Add aggregations
    )

# Exercise 5: Calculate derived features
# - recency: days between max_date and last_purchase_date
# - tenure: days between last_purchase_date and registration_date

customer_features = customer_features.withColumn("recency", ...) \
                                     .withColumn("tenure", ...)

display(customer_features)

## Section 5: Encoding Categorical Features

**Business context:** Countries are text (`USA`, `Germany`), but math requires numbers.

**Why StringIndexer?**
- Converts text → number: `USA` → 0, `Germany` → 1, `UK` → 2
- ML model can now use country as a feature
- Preserves information: different countries → different numbers

**Alternative approaches:**
- **OneHotEncoder:** USA → [1,0,0], Germany → [0,1,0] - better for some models
- **TargetEncoder:** Replace with segment probability - advanced technique

In [0]:
from pyspark.ml.feature import StringIndexer

# Exercise 6: Encode categorical features
# Use StringIndexer to convert 'country' to 'country_index'

indexer = # TODO: Create StringIndexer
# customer_features_encoded = ...

# display(customer_features_encoded)

## Section 6: Save Feature Table

**Business context:** This Customer 360 table is our "Gold" layer - ready for ML.

**What we're saving:**
- One row per customer
- Clean, validated data
- RFM features calculated
- Categorical features encoded

**Next step:** Workshop 3 will use this table to train the ML model.

In [0]:
# Exercise 7: Save the feature table as 'workshop_customer_features'

# TODO: Save table

## Best Practices: Data Cleaning. 

| Practice | Description |
|----------|-------------|
| **Clean before split** | Remove invalid data before train/test split |
| **Document decisions** | Record why you removed/imputed each column |
| **Preserve information** | Create missing flags before filling nulls |
| **Validate counts** | Check row counts after each cleaning step |
| **Avoid data leakage** | Calculate statistics on training data only |

---

# Solutions

Reference solutions for the exercises above.Compare your results.

In [0]:
# Load Data
df = spark.table("workshop_sales_data")

from pyspark.sql.functions import max, datediff, lit, col, sum, count

# 1. Filter
df_clean = df.filter((col("quantity") > 0) & (col("total_amount") >= 0))

# 2. Nulls & Duplicates
df_clean = df_clean.dropna(subset=["customer_id", "order_id","customer_segment"]) \
                   .fillna({"email": "unknown@example.com"}) \
                   .dropDuplicates()

# 3. Aggregation (RFM + Tenure)
from pyspark.sql.functions import max, datediff, lit, col

max_date_row = df_clean.select(max("order_datetime")).collect()
max_date = max_date_row[0][0]

customer_features = df_clean.groupBy("customer_id", "customer_segment", "country", "registration_date") \
    .agg(
        sum("total_amount").alias("total_spend"),
        count("order_id").alias("order_count"),
        max("order_datetime").alias("last_purchase_date")
    ) \
    .withColumn("recency", datediff(lit(max_date), col("last_purchase_date"))) \
    .withColumn("tenure", datediff(col("last_purchase_date"), col("registration_date")))

# 4. Encoding
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="country", outputCol="country_index", handleInvalid='keep')
customer_features_encoded = indexer.fit(customer_features).transform(customer_features)

# 5. Save
customer_features_encoded.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("workshop_customer_features")
display(customer_features_encoded)
